<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [74]:
%pip list

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

Package                       Version
----------------------------- ------------
absl-py                       1.4.0
asttokens                     2.4.0
astunparse                    1.6.3
attrs                         23.1.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.5
cachetools                    5.3.1
certifi                       2023.7.22
charset-normalizer            3.2.0
cmake                         3.27.4.1
comm                          0.1.4
contourpy                     1.1.0
cycler                        0.11.0
debugpy                       1.7.0
decorator                     5.1.1
emoji                         2.8.0
exceptiongroup                1.1.3
executing                     1.2.0
fastjsonschema                2.18.0
filelock                      3.12.3
flatbuffers                   23.5.26
fonttools                     4.42.1
gast                          0.4.0
gensim                        4.3.2
google-auth                   2.22.0
goog

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [75]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [76]:
# Posibles bandas
os.listdir("./songs_dataset/")

['dickinson.txt',
 'blink-182.txt',
 'britney-spears.txt',
 'adele.txt',
 'michael-jackson.txt',
 'dj-khaled.txt',
 'alicia-keys.txt',
 'bob-dylan.txt',
 'lady-gaga.txt',
 'r-kelly.txt',
 'missy-elliott.txt',
 'joni-mitchell.txt',
 'ludacris.txt',
 'jimi-hendrix.txt',
 'johnny-cash.txt',
 'bjork.txt',
 'bruce-springsteen.txt',
 'lorde.txt',
 'nickelback.txt',
 'nirvana.txt',
 'disney.txt',
 'nicki-minaj.txt',
 'nursery_rhymes.txt',
 'paul-simon.txt',
 'janisjoplin.txt',
 'patti-smith.txt',
 'cake.txt',
 'beatles.txt',
 'leonard-cohen.txt',
 'rihanna.txt',
 'bruno-mars.txt',
 'amy-winehouse.txt',
 'bieber.txt',
 'notorious-big.txt',
 'kanye.txt',
 'drake.txt',
 'lil-wayne.txt',
 'kanye-west.txt',
 'dolly-parton.txt',
 'bob-marley.txt',
 'notorious_big.txt',
 'radiohead.txt',
 'dr-seuss.txt',
 'eminem.txt',
 'lin-manuel-miranda.txt',
 'al-green.txt',
 'Kanye_West.txt',
 'Lil_Wayne.txt',
 'prince.txt']

In [77]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/lin-manuel-miranda.txt', sep='/n', header=None)
df.head()

/tmp/ipykernel_70675/9091060.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,"How does a bastard, orphan, son of a whore"
1,"And a Scotsman, dropped in the middle of a for..."
2,"In squalor, grow up to be a hero and a scholar..."
3,Got a lot farther by working a lot harder
4,By being a lot smarter By being a self-starter


In [78]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1084


### 1 - Preprocesamiento

In [79]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [80]:
# Demos un vistazo
sentence_tokens[:2]

[['how', 'does', 'a', 'bastard', 'orphan', 'son', 'of', 'a', 'whore'],
 ['and',
  'a',
  'scotsman',
  'dropped',
  'in',
  'the',
  'middle',
  'of',
  'a',
  'forgotten',
  'spot',
  'in',
  'the',
  'caribbean',
  'by',
  'providence',
  'impoverished']]

### 2 - Crear los vectores (word2vec)

In [81]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [82]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [83]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [84]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1084


In [85]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 377


### 3 - Entrenar embeddings

In [86]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 106246.171875
Loss after epoch 1: 48196.25
Loss after epoch 2: 47919.015625
Loss after epoch 3: 47397.5
Loss after epoch 4: 46953.875
Loss after epoch 5: 46654.125
Loss after epoch 6: 46838.78125


Loss after epoch 7: 46243.65625
Loss after epoch 8: 45127.15625
Loss after epoch 9: 44853.65625
Loss after epoch 10: 44954.0625
Loss after epoch 11: 44544.0
Loss after epoch 12: 43379.5
Loss after epoch 13: 42164.6875
Loss after epoch 14: 42546.4375
Loss after epoch 15: 42176.3125
Loss after epoch 16: 42370.125
Loss after epoch 17: 42268.0
Loss after epoch 18: 42837.375
Loss after epoch 19: 42812.6875


(105712, 225240)

### 4 - Ensayar

Para poder entender el siguiente analisis es necesario tener un contexto de la trama del musical en cuestion

**Acto 1**:

El musical comienza con Alexander Hamilton, un inmigrante caribeño, llegando a Nueva York en 1776. Hamilton, un joven ambicioso, se une al ejército revolucionario de George Washington durante la Guerra de Independencia de los Estados Unidos. Rápidamente se destaca por su inteligencia y se convierte en el ayudante de Washington.

Durante este tiempo, Hamilton se encuentra con varios personajes clave, incluyendo a Aaron Burr, quien se convierte en su amigo y rival. También conoce a las hermanas Schuyler: Eliza, con quien se casa, y Angelica, con quien tiene una conexión intensa pero no puede estar con ella debido a su compromiso con Eliza.

Hamilton juega un papel importante en la victoria de los Estados Unidos en la guerra y se convierte en uno de los autores de los "Artículos de la Confederación", que establecen la base para el gobierno de la nueva nación.

**Acto 2**:

El Acto 2 se centra en los primeros años de los Estados Unidos como una nación independiente y en la creación de la Constitución. Hamilton desempeña un papel fundamental en la redacción de la Constitución y en la promoción de su adopción a través de una serie de ensayos conocidos como "Los Federalistas".

Thomas Jefferson regresa a los Estados Unidos desde Francia y se convierte en el Secretario de Estado. Jefferson y Hamilton tienen visiones políticas opuestas, lo que lleva a intensos enfrentamientos.

El Acto 2 también explora la relación de Hamilton con su familia y los desafíos que enfrenta, incluyendo el escándalo de su affair con Maria Reynolds, que pone en peligro su reputación.

**Climax y Trágico Final**:

La rivalidad política entre Hamilton y Burr alcanza su punto culminante en un duelo fatal en 1804. Hamilton muere a manos de Burr, lo que pone fin a su historia.

La obra concluye con un epílogo que destaca el legado de Hamilton y cómo su influencia perdura en la historia de los Estados Unidos.

"Hamilton" es un musical lleno de música hip-hop y ritmos contemporáneos que cuenta la historia de uno de los Padres Fundadores de los Estados Unidos a través de una lente moderna y diversa. Explora temas de ambición, legado, amor y traición en el contexto de la formación de una nación. La obra ha sido elogiada por su innovación y ha dejado una marca significativa en el mundo del teatro y la cultura popular.

In [99]:
word = "hamilton"
print(w2v_model.wv.most_similar(positive=[word], topn=10))
print(w2v_model.wv.most_similar(negative=[word], topn=10))

[('name', 0.9665425419807434), ('alexander', 0.9294832944869995), ('back', 0.9149143099784851), ('hand', 0.906218945980072), ('same', 0.8789875507354736), ('mean', 0.8750167489051819), ('land', 0.872408390045166), ('man', 0.8719578981399536), ("what's", 0.8718845248222351), ('right', 0.8706536293029785)]
[('take', -0.33821240067481995), ('shot', -0.3517768085002899), ('time', -0.354216605424881), ('whoa', -0.35579976439476013), ('rise', -0.37545156478881836), ("it's", -0.3827356994152069), ('takes', -0.38384342193603516), ('scrappy', -0.38751041889190674), ('up', -0.3887084126472473), ('throwing', -0.3982303738594055)]


Positivas: las primeras palabras estas asociadas a la primera cancion del musical, donde el personaje repite "my name is alexander hamilton" varias veces. La pregunta "what's right" es una que se le hace el personaje a lo largo de todo el musical, por lo que tiene sentido que este ascociado a su nombre.

Negativas: parece que la mayoria son palabras que se usan poco a lo largo del musical, porque aparecen para los otros personajes tambien

In [101]:
word = "burr"
print(w2v_model.wv.most_similar(positive=[word], topn=10))
print(w2v_model.wv.most_similar(negative=[word], topn=10))

[('aaron', 0.9756542444229126), ('nothing', 0.9708666801452637), ('talk', 0.9703153371810913), ('against', 0.9650269150733948), ('sure', 0.9645829796791077), ('sir', 0.9633787870407104), ('stand', 0.9591177701950073), ('america', 0.958432137966156), ('yes', 0.9581354856491089), ('check', 0.9575738906860352)]
[('hey', -0.4454285800457001), ('yo', -0.4465112090110779), ('country', -0.45028606057167053), ('just', -0.48148372769355774), ('throwing', -0.4847804009914398), ('away', -0.48649197816848755), ('scrappy', -0.49305227398872375), ('young', -0.4971821904182434), ("i'm", -0.5065277218818665), ('like', -0.5161053538322449)]


Positivas: el personaje de Aaron Burr a lo largo de todo el musical enfatiza como el nunca quiere decir nada ("nothing") ni estar en contra ("against") de nada.

Negativas: otra vez no parece tener una relacion con el personaje especificamente.

In [100]:
word = "eliza"
print(w2v_model.wv.most_similar(positive=[word], topn=10))
print(w2v_model.wv.most_similar(negative=[word], topn=10))

[('tell', 0.9908677935600281), ("won't", 0.9899544715881348), ('lead', 0.9895618557929993), ('why', 0.9891394376754761), ('financial', 0.989048957824707), ('turns', 0.9883782267570496), ('eye', 0.9881983995437622), ('many', 0.9875673651695251), ('does', 0.9874441027641296), ("i've", 0.9870262145996094)]
[('takes', -0.5780946612358093), ('hey', -0.5923123955726624), ('wait', -0.5972515940666199), ('scrappy', -0.6026186347007751), ('yo', -0.605708122253418), ('away', -0.6103274822235107), ('young', -0.6114529371261597), ('country', -0.6148903965950012), ('shot', -0.6157116889953613), ('throwing', -0.6171424388885498)]


Positivas: Eliza es un personaje al que siempre se le esta escondiendo algo (la infidelidad de su marido con su hermana y con otra mujer, los duelos que terminan con la vida de su hijo y marido, etc). Tiene mucho sentido que se asocian las palabras "won't tell" con este personaje. Tambien es un personaje ligado al ascenso social, por lo que es lógico que el concepto de lo financiero ("financial") este relacionado con ella.

Negativas: mismo escenario, parece que son palabras que no estan muy asociadas a ningun personaje en particular

In [90]:
word = "angelica"
print(w2v_model.wv.most_similar(positive=[word], topn=10))
print(w2v_model.wv.most_similar(negative=[word], topn=10))

[('she', 0.9880316853523254), ('ay', 0.9781311750411987), ('france', 0.9719031453132629), ('where', 0.968976616859436), ('on', 0.9686877131462097), ('some', 0.9649363160133362), ("y'all", 0.9577064514160156), ('mother', 0.9572803974151611), ('ha', 0.957254946231842), ('from', 0.9568611979484558)]
[('throwing', -0.4863385260105133), ('away', -0.49111002683639526), ('wait', -0.5073025822639465), ('not', -0.5104750990867615), ('shot', -0.5163512229919434), ('am', -0.520524263381958), ('scrappy', -0.5259639024734497), ('young', -0.5357979536056519), ('takes', -0.5379138588905334), ('just', -0.5411667823791504)]


Positivas: Angelica se muda a Francia a la mitad del segundo acto.

Negativas: mismo escenario, parece que son palabras que no estan muy asociadas a ningun personaje en particular

### 5 - Visualizar agrupación de vectores

In [94]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [150]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=1000
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
# fig.show(renderer="colab") # esto para plotly en colab

In [151]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
# fig.show(renderer="colab") # esto para plotly en colab

Algunos aspectos interesantes:

- Las palabras revolution, future,  nation, now y free estan agrupadas. Los embeddings logran representar la trama de la revolucion por la independencia nacional y el pedido de que sea ahora.
- Otro ejemplo es france y hope, donde a lo largo de todo el musical se habla de francia como la esperanza para dar vuelta el resultado en la guerra
- Algunas canciones y las ideas que repiten tambien se encuentran claramente representadas: "I'm not throwing away my shot" y "just like my country" y "young, scruffy and hungry" son frases que se repiten varias veces en una de las canciones principales. Esto sucede con otras canciones que tienen mucha repeticion.
- Tambien se logran agrupar conceptos genericos como number y todos los numeros del 1 al 10.

Es interesante notar que como todas las canciones estan contando diferentes etapas y visiones de la misma historia hay obviamente clusters asociados a la repeticion de las canciones pero hay otros clusters asociados a las ideas principales relatadas en la historia, ya sea de forma general (el concepto de la revolucion) o especifica (los numeros se asocian a los duelos, por lo pasos que dan los duelistas).

In [149]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)